In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv


In [2]:
df = pd.read_csv('/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv', engine='python')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [3]:
df.shape

(550, 7)

In [4]:
df.dtypes

Name            object
Author          object
User Rating    float64
Reviews          int64
Price            int64
Year             int64
Genre           object
dtype: object

In [16]:
df.describe()

,user_rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


In [5]:
df = df.rename(columns={'User Rating': 'user_rating'})

In [6]:
df.head()

,Name,Author,user_rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [8]:
top_5_author = ( 
    df.groupby('Author', as_index=False)
    .agg({'Price':'sum'})
    .sort_values('Price', ascending=False)
    .rename(columns={'Price': 'sum_price'})[:5]
)

In [9]:
top_5_author

,Author,sum_price
11,American Psychological Association,460
230,The College Board,236
10,American Psychiatric Association,210
92,Gary Chapman,189
90,Gallup,153


In [10]:
top_5_user_rating = (
    df.groupby('Author', as_index=False)
    .agg({'user_rating':'sum'})
    .sort_values('user_rating', ascending=False)[:5]
)

In [11]:
top_5_user_rating

,Author,user_rating
120,Jeff Kinney,57.6
197,Rick Riordan,52.5
92,Gary Chapman,52.1
226,Suzanne Collins,51.3
11,American Psychological Association,45.0


In [12]:
df.query('Author == "Gary Chapman"').value_counts('Name')

Name
The 5 Love Languages: The Secret to Love that Lasts                          5
The 5 Love Languages: The Secret to Love That Lasts                          5
The Five Love Languages: How to Express Heartfelt Commitment to Your Mate    1
dtype: int64

In [13]:
df.Genre.value_counts()

Non Fiction    310
Fiction        240
Name: Genre, dtype: int64

In [14]:
whats_best_cost_genre= (
    df.groupby('Genre', as_index=False)
    .agg({'Price':'sum'})
    .sort_values('Price', ascending=False)
    .rename(columns={'Price':'sum_price_genre'})
)

In [15]:
whats_best_cost_genre

,Genre,sum_price_genre
1,Non Fiction,4601
0,Fiction,2604


In [17]:
minimum_rating = (
    df.groupby(['Author','Name', 'Price'], as_index=False)
    .agg({'user_rating':'sum'})
    .sort_values('user_rating', ascending=True).rename(columns={'user_rating':'user_rating_min'})[:5]
)
minimum_rating

,Author,Name,Price,user_rating_min
154,J.K. Rowling,The Casual Vacancy,12,3.3
135,Harper Lee,Go Set a Watchman: A Novel,19,3.6
350,Veronica Roth,Allegiant,13,3.9
128,Gillian Flynn,Gone Girl,9,4.0
240,Muriel Barbery,The Elegance of the Hedgehog,11,4.0


In [18]:
max_price = (
    df.groupby(['Author', 'Name', 'Genre', 'user_rating'], as_index=False)
    .agg({'Price':'max'}).sort_values('Price', ascending=False)[:5]
)
max_price

,Author,Name,Genre,user_rating,Price
10,American Psychiatric Association,Diagnostic and Statistical Manual of Mental Di...,Non Fiction,4.5,105
311,Stephenie Meyer,The Twilight Saga Collection,Fiction,4.7,82
204,Lin-Manuel Miranda,Hamilton: The Revolution,Non Fiction,4.9,54
33,Bill Simmons,The Book of Basketball: The NBA According to T...,Non Fiction,4.7,53
145,J. K. Rowling,Harry Potter Paperback Box Set (Books 1-7),Fiction,4.8,52
